In [9]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import curve_fit


def exponential_decay(TE, S0, T2_star):
    return S0 * np.exp(-TE / T2_star)

In [10]:

def load_matlab_data(file_path):
    """
    Load MATLAB data from a .mat file into Python.

    Parameters:
    - file_path: Path to the .mat file.

    Returns:
    - data: Dictionary with variable names from MATLAB as keys and loaded matrices as values.
    """
    data = scipy.io.loadmat(file_path)
    return data


In [11]:

def perform_t2_star_mapping_3d(img_data, echo_times):
    """
    Perform T2* mapping on 3D MRI data.

    Parameters:
    - img_data: 4D numpy array with shape (rows, columns, slices, echoTimes)
    - echo_times: 1D numpy array containing the echo times (TE) in milliseconds.

    Returns:
    - T2_star_map: 3D numpy array with shape (rows, columns, slices) containing the T2* values.
    """
    # Initialize the T2* map
    rows, cols, slices, _ = img_data.shape
    T2_star_map = np.zeros((rows, cols, slices))
    
    # Iterate over each slice and pixel to fit the signal decay
    for k in range(slices):
        for i in range(rows):
            for j in range(cols):
                signal_decay = img_data[i, j, k, :].astype(float)
                
                if np.all(signal_decay == 0):
                    continue
                
                try:
                    # Fit the signal decay to the exponential model
                    popt, _ = curve_fit(exponential_decay, echo_times, signal_decay, p0=(signal_decay[0], 50))
                    T2_star_map[i, j, k] = popt[1]  # T2* value is the second parameter
                except RuntimeError:
                    # If the fitting fails, set T2* to zero (or could use NaN)
                    T2_star_map[i, j, k] = 0
    
    # Handle negative and unrealistic T2* values
    T2_star_map[T2_star_map < 0] = 0
    
    return T2_star_map


In [12]:

def display_slice(T2_star_map, slice_index):
    """
    Display a single slice from the T2* map.

    Parameters:
    - T2_star_map: 3D numpy array with the T2* values.
    - slice_index: Index of the slice to display.
    """
    plt.figure()
    plt.imshow(T2_star_map[:, :, slice_index], cmap='jet')
    plt.colorbar()
    plt.title(f'T2* Map of Slice {slice_index}')
    plt.xlabel('Pixel X Coordinate')
    plt.ylabel('Pixel Y Coordinate')
    plt.show()


In [14]:
# Example usage
file_path = '/Users/tinghuili/Downloads/combined_data.mat'
data = load_matlab_data(file_path)

# Access a specific variable (e.g., 'your_variable_name') from the loaded data
your_variable = data['combinedData']
echo_times = np.array([0.27, 1.19, 2.19, 3.08])

In [15]:
perform_t2_star_mapping_3d(your_variable, echo_times)

/Users/tinghuili/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/var/folders/yf/qd0y3f9j4t94j0mb0fq1xfwc0000gn/T/ipykernel_2685/455902646.py:8: RuntimeWarning: overflow encountered in exp
  return S0 * np.exp(-TE / T2_star)


KeyboardInterrupt: 